## Exporting CSVs from the FairNow APIs

#### This notebook demonstrates how to generate CVSs from the FairNow APIs. The current supported outputs are 1. Severity/Probability Export, 2. Application Inventory Export, and 3. Compliance Export.

### Prerequisites

#### To use this notebook, you'll need a `Client ID` and `Client Secret`. These will either have been provided to you, or you can generate from https://app.fairnow.ai and going the the Admin menu. This notebook assumes you have these available to enter when prompted.

In [2]:
import json
from getpass import getpass
import httpx
from httpx_auth import OAuth2ClientCredentials

client_id = "2726p9mj2ghit00n23tkjatp84" # Replace with your Client ID
client_secret = getpass("Client Secret")
fairnow_token_endpoint = "https://auth.fairnow.dev/oauth2/token"

auth = OAuth2ClientCredentials(
    token_url=fairnow_token_endpoint,
    client_id=client_id,
    client_secret=client_secret,
)

fairnow_base_url = "https://api.fairnow.dev/v2"
client = httpx.Client(base_url=fairnow_base_url, auth=auth)

## Applications API call

In [17]:
application_route = "/applications"

apps_response = None
try:
    response = client.get(application_route, timeout=None)
    if response.status_code == 200:
        apps_response = response.json()
        print(f"Successfully retrieved applications")
        # Add this to see the structure of the response
        # print("\nResponse structure:")
        # print(json.dumps(response_data, indent=2)[:1000])  # Print firs
    else:
        print(f"Error: {response.status_code} - {response.text}")
except Exception as e:
    print(e)


Successfully retrieved applications


## Frameworks Control Call

## Pull Application Risk

In [11]:
import pandas as pd

# Extract fields from response
apps_data = []
for app in apps_response['applications']:
    app_id = app['application_id']
    app_name = app['application_name']
    # Handle cases where risk_item_list might be None or empty
    risk_items = app.get('risk_item_list', []) or []
    for risk_item in risk_items:
        apps_data.append({
            'application_id': app_id,
            'application_name': app_name,
            'risk_type': risk_item.get('risk_type', ''),
            'risk_type_label': risk_item.get('risk_type_label', ''),
            'severity': risk_item.get('severity', ''),
            'probability': risk_item.get('probability', ''),
            # 'description': risk_item.get('description', ''),
            # 'tool_tip': risk_item.get('tool_tip', '')
        })

# Convert to DataFrame
apps_df = pd.DataFrame(apps_data)
apps_df = apps_df.drop_duplicates()
print(f"Extracted risk items from applications")
apps_df.head()

Extracted 7 risk items from 24 applications


,application_id,application_name,risk_type,risk_type_label,severity,probability
0,06f5a5d3-3d39-4f24-aa24-5d288162c11d,Steve Test,performance,Performance,Low,Low
1,7148d64d-56cd-4aaa-b6f5-aa353b856d4c,Demo Application,safety,Safety,Low,Medium
2,7148d64d-56cd-4aaa-b6f5-aa353b856d4c,Demo Application,bias,Bias,High,High
3,7148d64d-56cd-4aaa-b6f5-aa353b856d4c,Demo Application,security,Security,High,High
4,7148d64d-56cd-4aaa-b6f5-aa353b856d4c,Demo Application,vendor,Vendor Dependency,Low,Low


### Exporting to TSV

Finally, let's export the data to a TSV file.

In [12]:
# Export to TSV
output_file = 'application_risks.tsv'
apps_df.to_csv(output_file, sep='\t', index=False)
print(f"Data exported to {output_file}")

Data exported to application_risks.tsv


## Frameworks

In [21]:
def get_framework_controls(framework_id):

    framework_response = None
    query_parameters = {"framework_id": framework_id}
    
    try:
        response = client.get(f"/controls/framework/", params=query_parameters, timeout=None)
        if response.status_code == 200:
            framework_response = response.json()
            print(f"Successfully retrieved framework controls")
            return framework_response
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(e)
        return None

In [19]:
import pandas as pd

# Extract fields from response
extracted_data = []
for app in apps_response['applications']:
    app_id = app['application_id']
    app_name = app['application_name']
    # Handle cases where risk_item_list might be None or empty
    # risk_assessment = app.get('risk_assessment', {})
    framework_items = risk_assessment.get('framework_assessment_items', []) or []
    for framework in framework_items:
        extracted_data.append({
            'application_id': app_id,
            'application_name': app_name,
            'framework_id': framework.get('framework_id', ''),
            'framework_name': framework.get('framework_name', ''),
        })

# Convert to DataFrame
df = pd.DataFrame(extracted_data)
df = df.drop_duplicates()
print(f"Extracted frameworks")

# Combine with framework controls
# Process controls for each framework
for framework_id in df['framework_id'].unique():
    controls = get_framework_controls(framework_id)
    if controls:
        # Process the controls data here
        print(f"Processing controls for {framework_id}")
        # Add your processing logic here
df.head()

Extracted frameworks
Error: 404 - {"detail":"Not Found"}
Error: 404 - {"detail":"Not Found"}
Error: 404 - {"detail":"Not Found"}
Error: 404 - {"detail":"Not Found"}
Error: 404 - {"detail":"Not Found"}
Error: 404 - {"detail":"Not Found"}
Error: 404 - {"detail":"Not Found"}
Error: 404 - {"detail":"Not Found"}
Error: 404 - {"detail":"Not Found"}
Error: 404 - {"detail":"Not Found"}


,application_id,application_name,framework_id,framework_name
0,05e0d38a-f18c-45ac-8eaf-216690f69e8d,Steve Test [renamed x2],colorado-sb-205-developer,Colorado AI Act - Developer
1,05e0d38a-f18c-45ac-8eaf-216690f69e8d,Steve Test [renamed x2],euact-gpai,EU AI Act - General Purpose AI
2,05e0d38a-f18c-45ac-8eaf-216690f69e8d,Steve Test [renamed x2],illinois-hb3773,Illinois HB 3773
3,05e0d38a-f18c-45ac-8eaf-216690f69e8d,Steve Test [renamed x2],maryland-hb-1202,Maryland HB1202 on Facial Recognition
4,05e0d38a-f18c-45ac-8eaf-216690f69e8d,Steve Test [renamed x2],utah-ai-policy-act,Utah AI Policy Act


Retrieve controls info